In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

# Baseline Model

In [2]:
def model_setup():
    
    m = Model("Callcenter")
    
    # Create variables
    x = m.addVars(M, N, name="x")
    
    # Set objective
    m.setObjective( quicksum(cost[i,j]*x[i,j] for i in range(M) for j in range(N)), GRB.MINIMIZE)
    
    # Experienced Manpower Constraint(only 70% of the 30 headcount for experienced staff is 
    # used for call center operations)
    m.addConstrs(quicksum(x[i,j] for i in range (0,1) for j in range(N)) == 21 for i in range(N))
    
    # Maximum capacity at the contact centre constraint
    m.addConstr(quicksum(x[i,j] for i in range(M) for j in range(N)) <= 200, "Max headcount capacity")
    
    # Hourly constraints
    m.addConstrs(( quicksum(cph[i,0]*x[i,0] for i in range(M)) >= 0.5 * demand[j] for j in range(9)), "type 1 calls")
    m.addConstrs(( quicksum(cph[i,1]*x[i,1] for i in range(M)) >= 0.2 * demand[j] for j in range(9)), "type 2 calls")
    m.addConstrs(( quicksum(cph[i,2]*x[i,2] for i in range(M)) >= 0.2 * demand[j] for j in range(9)), "type 3 calls")
    m.addConstrs(( quicksum(cph[i,3]*x[i,3] for i in range(M)) >= 0.1 * demand[j] for j in range(9)), "type 4 calls")
    
    m.setParam( 'OutputFlag', False)
    
    return m

In [3]:
### Parameters Set-up ###

# cost per employee - experienced agents cost 40 per hour, temp agents cost 10 per hour
cost = np.array([[40, 40, 40, 40],
                [10, 10, 10000, 10000],
                [10, 10000, 10, 10000],
                [10, 10000, 10000, 10],
                [10000, 10, 10, 10000],
                [10000, 10, 10000, 10],
                [10000, 10000, 10, 10]])

# demand - normally distributed with the following mean and variance
mean_d = np.array([170, 250, 310, 290, 220, 270, 310, 380, 300])

cov_d = np.array([[998.56, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 566.44, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1568.16, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 249.64, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1730.56, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 2530.09, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1232.01, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 4900, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 734.41]])

# calls per hour that can be taken by a certain officer i,j
# officers that cannot take a particular type of call can take zero calls of that type
cph = np.array([[6, 6, 6, 6],
                [4.5, 4.5, 0, 0],
                [4.5, 0, 4.5, 0],
                [4.5, 0, 0, 4.5],
                [0, 4.5, 4.5, 0],
                [0, 4.5, 0, 4.5],
                [0, 0, 4.5, 4.5]])

M, N = cost.shape

Sample_Size = 1000

cc_cost = np.zeros(Sample_Size)

np.random.seed(1988)
for i in range(Sample_Size):
    demand = np.minimum(np.maximum(np.random.multivariate_normal(mean_d, cov_d), 0), 500)
    m = model_setup()
    m.optimize()
    cc_cost[i] = m.objVal
    
print(demand)
    
avg_cc_cost = np.average(cc_cost)

Academic license - for non-commercial use only
[164.58413285 230.94202189 348.81424637 307.0440906  248.26055662
 330.58057554 292.86675365 365.76323557 329.60008708]


In [4]:
# average call center cost plus the 9 experienced agents that handle other administrative matters
print('Average manpower cost of operating call center per hour: ', avg_cc_cost + 9 * 40)
print('Average manpower cost of operating call center per day (9 hour workday): ', (avg_cc_cost + 9 * 40)* 9)

for v in m.getVars():
    if v.x > 0:
        #print(v.VarName[2], v.VarName[4], v.x)
        print("\nNumber of officers of type %g required to answer calls of type %g: %g" % (int(v.VarName[2])+1, int(v.VarName[4])+1, v.x))

Average manpower cost of operating call center per hour:  1792.137941776398
Average manpower cost of operating call center per day (9 hour workday):  16129.241475987583

Number of officers of type 1 required to answer calls of type 2: 2.71184

Number of officers of type 1 required to answer calls of type 3: 12.1921

Number of officers of type 1 required to answer calls of type 4: 6.09605

Number of officers of type 3 required to answer calls of type 1: 40.6404

Number of officers of type 5 required to answer calls of type 2: 12.6404


# Baseline model with 20% absenteeism & 5% downtime

In [5]:
def model_setup_absent():
    
    a = Model("Callcenter_absent")
    
    # Create variables
    x = a.addVars(M, N, name="x")
    
    # Set objective
    a.setObjective(quicksum(cost[i,j]*x[i,j] for i in range(M) for j in range(N)), GRB.MINIMIZE)
    
    # Experienced Manpower Constraint
    a.addConstrs((quicksum(x[i,j] for i in range (1) for j in range(N)) == 21 for i in range(N)), "expmanpower")
    
    # Maximum capacity at the contact centre constraint
    a.addConstr(quicksum(x[i,j] for i in range(M) for j in range(N)) <= 200, "Max headcount capacity")
    
    # Hourly constraints
    a.addConstrs(( quicksum(cph[i,0]*x[i,0] for i in range(M)) >= 0.5 * demand[j] for j in range(9)), "type 1 calls")
    a.addConstrs(( quicksum(cph[i,1]*x[i,1] for i in range(M)) >= 0.2 * demand[j] for j in range(9)), "type 2 calls")
    a.addConstrs(( quicksum(cph[i,2]*x[i,2] for i in range(M)) >= 0.2 * demand[j] for j in range(9)), "type 3 calls")
    a.addConstrs(( quicksum(cph[i,3]*x[i,3] for i in range(M)) >= 0.1 * demand[j] for j in range(9)), "type 4 calls")
    
    a.setParam( 'OutputFlag', False)
    
    return a

In [6]:
# for simplicity, scalar for absenteeism and downtime is multiplied into the call handling rate

cph = np.array([[6, 6, 6, 6],
                [4.5, 4.5, 0, 0],
                [4.5, 0, 4.5, 0],
                [4.5, 0, 0, 4.5],
                [0, 4.5, 4.5, 0],
                [0, 4.5, 0, 4.5],
                [0, 0, 4.5, 4.5]])

#absenteeism rate
absent = 0.20

#downtime rate
downtime = 0.05
cph = cph * (1-absent) * (1-downtime)

a = model_setup_absent()
a.optimize()


In [7]:
print('Average manpower cost of operating call center per hour: ', a.objVal + 9 * 40)
print('Average manpower cost of operating call center per day (9 hour workday): ', (a.objVal + 9 * 40)* 9)

for v in a.getVars():
    if v.x > 0:
        print("\nNumber of officers of type %g required to answer calls of type %g: %g" % (int(v.VarName[2])+1, int(v.VarName[4])+1, v.x))

Average manpower cost of operating call center per hour:  1989.4831449341111
Average manpower cost of operating call center per day (9 hour workday):  17905.348304407

Number of officers of type 1 required to answer calls of type 3: 12.9789

Number of officers of type 1 required to answer calls of type 4: 8.02112

Number of officers of type 3 required to answer calls of type 1: 53.4742

Number of officers of type 5 required to answer calls of type 2: 21.3897

Number of officers of type 5 required to answer calls of type 3: 4.08449


In [8]:
### Percentage of Missed calls over a 30 day period ###

#number of days
D = 30

mean_d = np.array([170, 250, 310, 290, 220, 270, 310, 380, 300])

cov_d = np.array([[998.56, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 566.44, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1568.16, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 249.64, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1730.56, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 2530.09, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1232.01, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 4900, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 734.41]])

missedcalls1 = np.zeros(D)
totaldemand1 = np.zeros(D)
missedcalls2 = np.zeros(D)
totaldemand2 = np.zeros(D)
missedcalls3 = np.zeros(D)
totaldemand3 = np.zeros(D)
missedcalls4 = np.zeros(D)
totaldemand4 = np.zeros(D)
missedcallstotal = np.zeros(D)

#type 1 total capacity
t1 = (1-absent)*(1-downtime)*round(53.4742 * 4.5, 0)
#type 2 total capacity
t2 = (1-absent)*(1-downtime)*round(21.3897 * 4.5, 0)
#type 3 total capacity
t3 = (1-absent)*(1-downtime)*round(12.9789 * 6 + 4.08449 * 4.5, 0)
#type 4 total capacity
t4 = (1-absent)*(1-downtime)*round(8.02112 * 6, 0)

np.random.seed(1234)

for d in range(D):
    demand = np.minimum(np.maximum(np.random.multivariate_normal(mean_d, cov_d), 0), 500)
    demand1 = demand * 0.5
    demand2 = demand * 0.2
    demand3 = demand * 0.2
    demand4 = demand * 0.1
    for i in range(9):
        totaldemand1[d] += demand1[i]
        totaldemand2[d] += demand2[i]
        totaldemand3[d] += demand3[i]
        totaldemand4[d] += demand4[i]
        #number of missed calls of type 1
        if demand1[i] - t1 > 0:
            missedcalls1[d] += demand1[i] - t1
        #number of missed calls of type 2
        if demand2[i] - t2 > 0:
            missedcalls2[d] += demand2[i] - t2
        #number of missed calls of type 3
        if demand3[i] - t3 > 0:
            missedcalls3[d] += demand3[i] - t3
        #number of missed calls of type 4
        if demand4[i] - t4 > 0:
            missedcalls4[d] += demand4[i] - t4
            
percentagemissed1 = (missedcalls1 / totaldemand1) * 100
percentagemissed2 = (missedcalls2 / totaldemand2) * 100
percentagemissed3 = (missedcalls3 / totaldemand3) * 100
percentagemissed4 = (missedcalls4 / totaldemand4) * 100

print('Percentage of Type 1 calls missed over 30 days:')
print()
print(percentagemissed1)
print('\nPercentage of Type 2 calls missed over 30 days:')
print()
print(percentagemissed2)
print('\nPercentage of Type 3 calls missed over 30 days:')
print()
print(percentagemissed3)
print('\nPercentage of Type 4 calls missed over 30 days:')   
print()
print(percentagemissed4)
print('\nTotal')
percentagetotal = percentagemissed1 + percentagemissed2 + percentagemissed3 + percentagemissed4
print(percentagetotal)
print('\nAverage percentage of calls dropped over a 30 day period:', round(np.average(percentagetotal), 4))

Percentage of Type 1 calls missed over 30 days:

[1.84402122 0.         4.17994963 0.         2.37161205 0.
 0.         0.96667183 3.29691175 0.         0.79231896 0.
 3.97807736 0.         0.         0.         3.5321066  0.40177394
 0.         0.         0.         0.         1.40034191 0.67405373
 2.49418067 0.         2.03396838 0.         0.         1.04976168]

Percentage of Type 2 calls missed over 30 days:

[1.90406588 0.         4.23979758 0.         2.4902474  0.
 0.         1.05193953 3.41322847 0.         0.85481972 0.
 4.038909   0.         0.         0.         3.58799851 0.46465907
 0.         0.         0.         0.         1.46319244 0.77604363
 2.55151562 0.         2.09216027 0.         0.         1.10662316]

Percentage of Type 3 calls missed over 30 days:

[1.90406588 0.         4.23979758 0.         2.4902474  0.
 0.         1.05193953 3.41322847 0.         0.85481972 0.
 4.038909   0.         0.         0.         3.58799851 0.46465907
 0.         0.         0. 

In [9]:
### Percentage of Missed calls over a 30 day x 12 month period ###

#number of days and number of months
D = 30
MN = 100

mean_d = np.array([170, 250, 310, 290, 220, 270, 310, 380, 300])

cov_d = np.array([[998.56, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 566.44, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1568.16, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 249.64, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1730.56, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 2530.09, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1232.01, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 4900, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 734.41]])

missedcalls1 = np.zeros(D)
totaldemand1 = np.zeros(D)
missedcalls2 = np.zeros(D)
totaldemand2 = np.zeros(D)
missedcalls3 = np.zeros(D)
totaldemand3 = np.zeros(D)
missedcalls4 = np.zeros(D)
totaldemand4 = np.zeros(D)
missedcallstotal = np.zeros(D)

#type 1 total capacity
t1 = (1-absent)*(1-downtime)*round(53.4742 * 4.5, 0)
#type 2 total capacity
t2 = (1-absent)*(1-downtime)*round(21.3897 * 4.5, 0)
#type 3 total capacity
t3 = (1-absent)*(1-downtime)*round(12.9789 * 6 + 4.08449 * 4.5, 0)
#type 4 total capacity
t4 = (1-absent)*(1-downtime)*round(8.02112 * 6, 0)

np.random.seed(2013)

for m in range(MN):
    for d in range(D):
        demand = np.minimum(np.maximum(np.random.multivariate_normal(mean_d, cov_d), 0), 500)
        demand1 = demand * 0.5
        demand2 = demand * 0.2
        demand3 = demand * 0.2
        demand4 = demand * 0.1
        for i in range(9):
            totaldemand1[d] += demand1[i]
            totaldemand2[d] += demand2[i]
            totaldemand3[d] += demand3[i]
            totaldemand4[d] += demand4[i]
            #number of missed calls of type 1
            if demand1[i] - t1 > 0:
                missedcalls1[d] += demand1[i] - t1
            #number of missed calls of type 2
            if demand2[i] - t2 > 0:
                missedcalls2[d] += demand2[i] - t2
            #number of missed calls of type 3
            if demand3[i] - t3 > 0:
                missedcalls3[d] += demand3[i] - t3
            #number of missed calls of type 4
            if demand4[i] - t4 > 0:
                missedcalls4[d] += demand4[i] - t4

percentagemissed1 = (missedcalls1 / totaldemand1) * 100
percentagemissed2 = (missedcalls2 / totaldemand2) * 100
percentagemissed3 = (missedcalls3 / totaldemand3) * 100
percentagemissed4 = (missedcalls4 / totaldemand4) * 100

print('Percentage of Type 1 calls missed over 30 days:')
print()
print(percentagemissed1)
print('\nPercentage of Type 2 calls missed over 30 days:')
print()
print(percentagemissed2)
print('\nPercentage of Type 3 calls missed over 30 days:')
print()
print(percentagemissed3)
print('\nPercentage of Type 4 calls missed over 30 days:')   
print()
print(percentagemissed4)
print('\nTotal')
percentagetotal = percentagemissed1 + percentagemissed2 + percentagemissed3 + percentagemissed4
print(percentagetotal)
print('\nAverage percentage of calls dropped over a 30 day period:', round(np.average(percentagetotal), 4))

Percentage of Type 1 calls missed over 30 days:

[1.13489728 1.36319859 1.26888818 1.48031376 1.02827724 1.39150924
 1.47539672 1.50647904 1.14362106 1.24596106 1.55557842 1.51562446
 1.3319472  1.22926431 1.31907711 1.37668006 1.56949843 1.37538412
 1.54104408 1.32764669 1.42604725 1.50630602 1.25627308 1.57707345
 1.47643866 1.2636965  1.69072358 1.60655466 1.29551704 1.58664851]

Percentage of Type 2 calls missed over 30 days:

[1.17563797 1.404935   1.31765321 1.5245462  1.06775463 1.43946492
 1.52198351 1.55169203 1.18402697 1.28507641 1.59760132 1.55994179
 1.37878508 1.26911498 1.36051081 1.41632317 1.61625154 1.42322987
 1.58949519 1.37372844 1.47192302 1.55509785 1.29783863 1.62254023
 1.52129197 1.31232538 1.74375088 1.65825745 1.34025759 1.63191965]

Percentage of Type 3 calls missed over 30 days:

[1.17563797 1.404935   1.31765321 1.5245462  1.06775463 1.43946492
 1.52198351 1.55169203 1.18402697 1.28507641 1.59760132 1.55994179
 1.37878508 1.26911498 1.36051081 1.41632317 

# Worst-case scanerio - meeting 95% of potential call volume with 20% absenteeism & 5% downtime

In [10]:
import scipy.stats
#set percentile of calls that must be taken
p = 0.95

mean_d = np.array([170, 250, 310, 290, 220, 270, 310, 380, 300])
sd_d = np.array([31.6, 23.8, 39.6, 15.8, 41.6, 50.3, 35.1, 70, 27.1])

# for simplicity, scalar for absenteeism and downtime is multiplied into the call handling rate
cph = np.array([[6, 6, 6, 6],
                [4.5, 4.5, 0, 0],
                [4.5, 0, 4.5, 0],
                [4.5, 0, 0, 4.5],
                [0, 4.5, 4.5, 0],
                [0, 4.5, 0, 4.5],
                [0, 0, 4.5, 4.5]])

#absenteeism rate
absent = 0.20

#downtime rate
downtime = 0.05
cph = cph * (1-absent) * (1-downtime)

highdemand = np.zeros(9)

for i in range(9):
    hdemand = np.maximum(scipy.stats.norm.ppf(p,mean_d[i],sd_d[i]),0)
    highdemand[i] = hdemand

print(highdemand)
highdemand = np.amax(highdemand)
print(highdemand)

[221.97737461 289.14751632 375.13620363 315.98868731 288.42591088
 352.73613744 367.73436231 495.13975389 344.57553329]
495.13975388660305


In [11]:
def model_setup_high_demand():
    
    n = Model("Callcenter_highdemand")
    
    # Create variables
    x = n.addVars(M, N, name="x")
    
    # Set objective
    n.setObjective( quicksum(cost[i,j]*x[i,j] for i in range(M) for j in range(N)), GRB.MINIMIZE)
    
    # Experienced Manpower Constraint
    n.addConstr((quicksum(x[i,j] for i in range(0,1) for j in range(N)) == 21), "experienced manpower" )
    
    # Maximum capacity at the contact centre constraint
    n.addConstr(quicksum(x[i,j] for i in range(M) for j in range(N)) <= 200, "Max headcount capacity")
    
    # Hourly constraints
    # Model must satify requirements for the hour of highest demand to be optimal for the entire day
    n.addConstr(( quicksum(cph[i,0]*x[i,0] for i in range(M)) >= 0.5 * highdemand), "type 1 calls")
    n.addConstr(( quicksum(cph[i,1]*x[i,1] for i in range(M)) >= 0.2 * highdemand), "type 2 calls")
    n.addConstr(( quicksum(cph[i,2]*x[i,2] for i in range(M)) >= 0.2 * highdemand), "type 3 calls")
    n.addConstr(( quicksum(cph[i,3]*x[i,3] for i in range(M)) >= 0.1 * highdemand), "type 4 calls")
    
    n.setParam( 'OutputFlag', False)
    
    return n

In [12]:
n = model_setup_high_demand()

n.optimize()

print('Average manpower cost of operating call center per hour: ', n.objVal + 9 * 40)
print('Average manpower cost of operating call center per day (9 hour workday): ', (n.objVal + 9 * 40)* 9)

for v in n.getVars():
    if v.x > 0:
        print("\nNumber of officers of type %g required to answer calls of type %g: %g" % (int(v.VarName[2])+1, int(v.VarName[4])+1, v.x))
        

Average manpower cost of operating call center per hour:  2367.7770581479617
Average manpower cost of operating call center per day (9 hour workday):  21309.993523331657

Number of officers of type 1 required to answer calls of type 3: 10.1417

Number of officers of type 1 required to answer calls of type 4: 10.8583

Number of officers of type 3 required to answer calls of type 1: 72.3889

Number of officers of type 5 required to answer calls of type 2: 28.9555

Number of officers of type 5 required to answer calls of type 3: 15.4333


# Sensitivity Analysis

In [13]:
Sen_constr = pd.DataFrame(columns=['Constraint Name', 'Shadow Price', 'Constraint Value', 'Slack', 'Upper Range' ,'Lower Range'])

print("\nConstraints Sensitivity Information:")

for d in n.getConstrs():
    #print(d.ConstrName, d.Pi, d.Slack, d.SARHSUP, d.SARHSLow)
    Sen_constr = Sen_constr.append(pd.Series([d.ConstrName, d.Pi, d.RHS, d.Slack, d.SARHSUP, d.SARHSLow], index=Sen_constr.columns ), ignore_index=True)

Sen_constr.set_index('Constraint Name',inplace=True)
Sen_constr


Constraints Sensitivity Information:


,Shadow Price,Constraint Value,Slack,Upper Range,Lower Range
Constraint Name,,,,,
experienced manpower,26.666667,21.000000,0.000000,3.257498e+01,10.858328
Max headcount capacity,0.000000,200.000000,62.222294,1.000000e+100,137.777706
type 1 calls,2.923977,247.569877,0.000000,4.603701e+02,-0.000000
type 2 calls,2.923977,99.027951,0.000000,3.118282e+02,-0.000000
type 3 calls,2.923977,99.027951,0.000000,3.118282e+02,46.246025
type 4 calls,2.923977,49.513975,0.000000,9.576000e+01,-0.000000


# Number of missed calls with manpower at high demand levels

In [14]:
#numberofdays
D = 30

mean_d = np.array([170, 250, 310, 290, 220, 270, 310, 380, 300])

cov_d = np.array([[998.56, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 566.44, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1568.16, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 249.64, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1730.56, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 2530.09, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1232.01, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 4900, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 734.41]])

missedcalls1 = np.zeros(D)
totaldemand1 = np.zeros(D)
missedcalls2 = np.zeros(D)
totaldemand2 = np.zeros(D)
missedcalls3 = np.zeros(D)
totaldemand3 = np.zeros(D)
missedcalls4 = np.zeros(D)
totaldemand4 = np.zeros(D)
missedcallstotal = np.zeros(D)

#type 1 total capacity
t1 = (1-absent)*(1-downtime)*round(72.3889 * 4.5, 0)
#type 2 total capacity
t2 = (1-absent)*(1-downtime)*round(28.9555 * 4.5, 0)
#type 3 total capacity
t3 = (1-absent)*(1-downtime)*round(10.1417 * 6 + 15.4333 * 4.5, 0)
#type 4 total capacity
t4 = (1-absent)*(1-downtime)*round(10.8583 * 6, 0)

np.random.seed(1988)
for d in range(D):
    demand = np.minimum(np.maximum(np.random.multivariate_normal(mean_d, cov_d), 0), 500)
    demand1 = demand * 0.5
    demand2 = demand * 0.2
    demand3 = demand * 0.2
    demand4 = demand * 0.1
    for i in range(9):
        totaldemand1[d] += demand1[i]
        totaldemand2[d] += demand2[i]
        totaldemand3[d] += demand3[i]
        totaldemand4[d] += demand4[i]
        #number of missed calls of type 1
        if demand1[i] - t1 > 0:
            missedcalls1[d] += demand1[i] - t1
        #number of missed calls of type 2
        if demand2[i] - t2 > 0:
            missedcalls2[d] += demand2[i] - t2
        #number of missed calls of type 3
        if demand3[i] - t3 > 0:
            missedcalls3[d] += demand3[i] - t3
        #number of missed calls of type 4
        if demand4[i] - t4 > 0:
            missedcalls4[d] += demand4[i] - t4

percentagemissed1 = (missedcalls1 / totaldemand1) * 100
percentagemissed2 = (missedcalls2 / totaldemand2) * 100
percentagemissed3 = (missedcalls3 / totaldemand3) * 100
percentagemissed4 = (missedcalls4 / totaldemand4) * 100

print('Percentage of Type 1 calls missed over 30 days:')
print()
print(percentagemissed1)
print('\nPercentage of Type 2 calls missed over 30 days:')
print()
print(percentagemissed2)
print('\nPercentage of Type 3 calls missed over 30 days:')
print()
print(percentagemissed3)
print('\nPercentage of Type 4 calls missed over 30 days:')   
print()
print(percentagemissed4)
print('\nTotal')
percentagetotal = percentagemissed1 + percentagemissed2 + percentagemissed3 + percentagemissed4
print(percentagetotal)
print('\nAverage percentage of calls dropped over a 30 day period:', round(np.average(percentagetotal), 4))

Percentage of Type 1 calls missed over 30 days:

[0.         0.         0.         0.         0.         0.16928633
 0.         0.         0.         0.         0.         0.15934544
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]

Percentage of Type 2 calls missed over 30 days:

[0.         0.         0.         0.         0.         0.22672277
 0.         0.         0.         0.         0.         0.21340906
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]

Percentage of Type 3 calls missed over 30 days:

[0.         0.         0.         0.         0.         0.22672277
 0.         0.         0.         0.         0.         0.21340906
 0.         0.         0.         0.         0.         0.
 0.         0.    